In [1]:
from PairTrading_eng import *
import pytz
from ib_insync import *
import datetime
import pandas_market_calendars as mcal

## Interactive Brokers Setting
### Account creataion and tws desktop offline version  installation
1. Account creation: https://www.interactivebrokers.com/en/home.php
2. tws (Interactive Brokers Trading Workstation) desktop Installation: https://www.interactivebrokers.com/en/trading/ibkr-desktop.php


In [2]:
try:
    if ib.isConnected():
        print('Connection is already established')
except NameError:
    util.startLoop()  # needed for Jupyter
    ib = IB()
    ib.connect()

In [3]:
accountSummary = pd.DataFrame(ib.accountSummary())
AvailableFunds = float(accountSummary[accountSummary["tag"] == "AvailableFunds"]["value"].iloc[0])
BuyingPower = float(accountSummary[accountSummary["tag"] == "BuyingPower"]["value"].iloc[0])

print(BuyingPower)

933097.37


## Importing Data from a Kaggle Notebook

### Preparation Steps

Before downloading data from a Kaggle notebook, ensure that the Kaggle API is installed and configured:

1. **Install the Kaggle API**:
   ```bash
   pip install kaggle
2. Download the kaggle.json file from the Kaggle account.
3. Place the kaggle.json file in the appropriate directory:
   - Mac/Linux: /username/.kaggle
   - Windows: C:\Users\Your-Username\\.kaggle
4. Should download files from kaggle output to local folder
   ```bash
   kaggle kernels output dtmanager1979/stock-trading-eda-scheduled -p /path/to/dest
5. The notebook can be found [here](https://www.kaggle.com/code/dtmanager1979/stock-trading-eda-scheduled/output). 
For instructional purposes, the algorithm has been slightly simplified, but custom algorithms can be created if necessary. 


In [4]:
# !kaggle kernels output dtmanager1979/stock-trading-eda-scheduled -p /Users/songyouk/PairsTrading
# !kaggle kernels output dtmanager1979/stock-trading-eda-scheduled -p C:\Users\DTman\onlinecourse\PairsTradingAutomation

In [5]:
if os.path.exists('../data/last_pairs_update.txt'):
    with open("../data/last_pairs_update.txt", 'r') as f:
        last_pairs_update = datetime.datetime.strptime(f.read(), '%Y-%m-%d %H:%M:%S.%f')
        
else:
    last_pairs_update = datetime.datetime.now() - datetime.timedelta(days=7) # set dummy last_pairs_update

time_diff = datetime.datetime.now() - last_pairs_update

# check if the last update is eariler than 5 days ago and also the day is Monday or Tuesday or Wednesday
if time_diff >= datetime.timedelta(days=5) and (datetime.datetime.now().weekday() in [0, 1, 2, 3, 4]): 
    # update the path based on your folder structure and your operating system. Note that in Windows, you need to specify the path with double backslashes.
    # if .. is not working, try with full path
    os.system('kaggle kernels output dtmanager1979/stock-trading-eda-scheduled -p ../')
    # os.system('kaggle kernels output dtmanager1979/stock-trading-eda-scheduled -p ..\\')
    last_pairs_update = datetime.datetime.now()
    with open("../data/last_pairs_update.txt", 'w') as f:
        f.write(str(last_pairs_update))

should pip install nbformat

### get the list of open stocks

In [6]:
df_sel= pd.read_pickle("../data/df_sel.pkl")[["median_window", "median_zscore_threshold"]]
# df_sel = pd.DataFrame(data)[["median_window", "median_zscore_threshold"]]
stocks = [stock for pair in df_sel.index for stock in pair]

In [7]:
df_sel

,median_window,median_zscore_threshold
pair,,
"(COR, NWSA)",20.0,2.05
"(ECL, INTU)",25.0,2.10


In [8]:
if len(df_sel) > 7:
    df_sel = df_sel.iloc[0:7,:]

In [9]:
positions = ib.positions()
df_current_positions = pd.DataFrame(positions)
stocks_open = []
if len(df_current_positions) == 0:
    print("No open positions")
else:
    df_current_positions['symbol'] = df_current_positions['contract'].apply(lambda x: x.symbol)
    stocks_open = df_current_positions['symbol'].to_list()
    print(stocks_open)

['NVDA']


## Close the stocks which are open but not included in new portfolio
If the open stocks is not included in stocks to trade, then close it

In [10]:
stocks_to_trade = [stock for pair in df_sel.index.to_list() for stock in pair]
stocks_to_close = [stock for stock in stocks_open if stock not in stocks_to_trade]
print(stocks_to_close)

['NVDA']


In [11]:
df_current_positions

,account,contract,position,avgCost,symbol
0,DU9209737,"Stock(conId=4815747, symbol='NVDA', exchange='...",50.0,134.45,NVDA


In [12]:
if len(df_current_positions) > 0:
    for index, row in df_current_positions.iterrows():
        stock = row['contract'].symbol
        position = row['position']
        if stock in stocks_to_close:
            print(f"Closing position for {stock}")
            contract = Stock(stock, 'SMART', 'USD')
            ib.qualifyContracts(contract)
            action = 'SELL' if position > 0 else 'BUY'
            order = MarketOrder(action, abs(position))
            trade = ib.placeOrder(contract, order)

Closing position for NVDA


it's ok if there is delay to close the positions since it does not interfere of new contract!

## Get the positions summary

In [13]:
def get_positions_summary(stocks_to_trade, df_pairs_wt_paras, df_whole_intraday, df_whole, ib, margin_ratio):


    accountSummary = pd.DataFrame(ib.accountSummary())
    NetLiquidation = float(accountSummary[accountSummary["tag"] == "NetLiquidation"]["value"].iloc[0])
    BuyingPower = float(accountSummary[accountSummary["tag"] == "BuyingPower"]["value"].iloc[0])
    
    
    margin_init = NetLiquidation/len(df_pairs_wt_paras) 
    ls_current_future_positions= [PairTradingUpdatePosition(df_whole_intraday = df_whole_intraday, 
                                                    df_whole = df_whole, 
                                                    margin_init = margin_init, 
                                                    margin_ratio = margin_ratio, 
                                                    df_pairs_wt_paras = df_pairs_wt_paras, 
                                                    ib = ib,
                                                    pair = pair).update_position_summary() for pair in df_sel.index]

    df_current_future_positions = pd.DataFrame(ls_current_future_positions)

    df_current_future_positions["stock1_order_numbers"] = df_current_future_positions["stock1_future_numbers"] - df_current_future_positions["stock1_current_numbers"]
    df_current_future_positions["stock2_order_numbers"] = df_current_future_positions["stock2_future_numbers"] - df_current_future_positions["stock2_current_numbers"]

    return df_current_future_positions

In [14]:
while True:
    if (len(stocks)>=1):
        break

## Realtime Trading

In [15]:
# Define the New York timezone
new_york_tz = pytz.timezone('America/New_York')

# Get current time in New York, subtract 10 days, and set time to 4 PM
dummy_trading_time = datetime.datetime.now(new_york_tz).replace(hour=16, minute=0, second=0, microsecond=0) - timedelta(days=7)

df_trading_time = df_sel.copy()
# apply the new column last_trading_time to yesterday's 4 PM based on new york time zone
df_trading_time['last_trading_time'] = dummy_trading_time

In [16]:
def ib_order_execute(df_current_future_positions, df_trading_time):
    for index, row in df_current_future_positions.iterrows():
        stock1 = row['pair'][0]
        stock2 = row['pair'][1]
        stock1_order_numbers = row['stock1_order_numbers']
        stock2_order_numbers = row['stock2_order_numbers']
        

        now = datetime.datetime.now(pytz.timezone('America/New_York'))
        start_time = now.replace(hour=9, minute=30, second=0, microsecond=0)
        end_time = now.replace(hour=16, minute=0, second=0, microsecond=0)
        
        last_trading_time = df_trading_time.loc[df_trading_time.index == (stock1, stock2), 'last_trading_time'].iloc[-1]
        if start_time <= now <= end_time:
            # Check if the last trading time is on the same day as the current day and time difference is at least 10 minutes
            if ((last_trading_time.date() == now.date()) and (now - last_trading_time) >= datetime.timedelta(minutes=60)) or (last_trading_time.date() != now.date()):
                
                
                if (stock1_order_numbers > 5) or (stock2_order_numbers < -5):
                    print(f"Buying {stock1_order_numbers} of {stock1}")
                    print(f"Selling {stock2_order_numbers} of {stock2}")  
                    contract_stock1 = Stock(stock1, 'SMART', 'USD')
                    ib.qualifyContracts(contract_stock1)
                    order_stock1 = MarketOrder('BUY', abs(stock1_order_numbers))
                    trade_stock1 = ib.placeOrder(contract_stock1, order_stock1)                
  
                    contract_stock2 = Stock(stock2, 'SMART', 'USD')
                    ib.qualifyContracts(contract_stock2)
                    order_stock2 = MarketOrder('SELL', abs(stock2_order_numbers))
                    trade_stock2 = ib.placeOrder(contract_stock2, order_stock2)
                    df_trading_time.loc[df_trading_time.index == (stock1, stock2), 'last_trading_time'] = datetime.datetime.now(pytz.timezone('America/New_York'))
                    
                elif (stock1_order_numbers < -5) or (stock2_order_numbers > 5):
                    print(f"Selling {stock1_order_numbers} of {stock1}")
                    print(f"Buying {stock2_order_numbers} of {stock2}")  
                    contract_stock1 = Stock(stock1, 'SMART', 'USD')
                    ib.qualifyContracts(contract_stock1)
                    order_stock1 = MarketOrder('SELL', abs(stock1_order_numbers))
                    trade_stock1 = ib.placeOrder(contract_stock1, order_stock1)                
  
                    contract_stock2 = Stock(stock2, 'SMART', 'USD')
                    ib.qualifyContracts(contract_stock2)
                    order_stock2 = MarketOrder('BUY', abs(stock2_order_numbers))
                    trade_stock2 = ib.placeOrder(contract_stock2, order_stock2)
                    df_trading_time.loc[df_trading_time.index == (stock1, stock2), 'last_trading_time'] = datetime.datetime.now(pytz.timezone('America/New_York'))

In [17]:
now = datetime.datetime.now(pytz.timezone('America/New_York'))
schedule = mcal.get_calendar('NYSE').schedule(start_date=now, end_date=now)
# if the schedule is empty, it is not a trading day so out from while loop
# if len(schedule) == 0 or now.hour > 16:
#     break
stocks = [stock for pair in df_sel.index for stock in pair]
data_1d_1y = yf.download(tickers = stocks, period="1y",interval="1d", progress = False)['Adj Close']
data_5m_60d = yf.download(tickers = stocks, period="60d",interval="5m", progress = False)['Adj Close']
df_current_future_positions = get_positions_summary(stocks_to_trade = stocks_to_trade, df_pairs_wt_paras = df_sel, df_whole_intraday = data_5m_60d, df_whole = data_1d_1y, ib = ib, 
                                                    margin_ratio = 0.25)

In [18]:
import pandas_market_calendars as mcal

while True:
    now = datetime.datetime.now(pytz.timezone('America/New_York'))
    schedule = mcal.get_calendar('NYSE').schedule(start_date=now, end_date=now)
    # if the schedule is empty, it is not a trading day so out from while loop
    if len(schedule) == 0 or now.hour > 16:
        break
    stocks = [stock for pair in df_sel.index for stock in pair]
    data_1d_1y = yf.download(tickers = stocks, period="1y",interval="1d", progress = False)['Adj Close']
    data_5m_60d = yf.download(tickers = stocks, period="60d",interval="5m", progress = False)['Adj Close']
    df_current_future_positions = get_positions_summary(stocks_to_trade = stocks_to_trade, df_pairs_wt_paras = df_sel, df_whole_intraday = data_5m_60d, df_whole = data_1d_1y, ib = ib, 
                                                        margin_ratio = 0.25)
    ib_order_execute(df_current_future_positions = df_current_future_positions, df_trading_time = df_trading_time)
    ib.sleep(60)
    # if the time is after 4 PM, break the loop
    if now.hour > 16:
        break